# ARCHS4 model building with PLIER2

## Load libraries

In [ ]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

if (!requireNamespace("PLIER2", quietly = TRUE)) {
    REPO_PATH <- "/home/msubirana/Documents/pivlab/PLIER2" 
    remotes::install_local(REPO_PATH, force = TRUE, dependencies = FALSE)
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER)
library(PLIER2)
library(hdf5r)
library(biomaRt)

source(here("config.R"))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/plier2-analyses

Loading required package: RColorBrewer

Loading required package: gplots


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: pheatmap

Loading required package: qvalue


Attaching package: ‘PLIER2’


The following objects are masked from ‘package:PLIER’:

    commonRows, num.pc, projectPLIER


The following object is masked from ‘package:bigstatsr’:

    AUC




## Output directory

In [2]:
output_dir <- config$ARCHS4$DATASET_FOLDER
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

In [ ]:
archs4_fbm_filt <- readRDS(file.path(output_dir, "archs4_fbm_filt.rds"))

## SVD computation and SVD K estimation

In [ ]:
g_fb <- nrow(archs4_fbm_filt)
samples_fb <- ncol(archs4_fbm_filt)
SVD_K_archs4  <- min(g_fb, samples_fb) - 1

message("Using SVD K = ", SVD_K_archs4)

set.seed(1)

N_CORES <- config$ARCHS4$PLIER_PARAMS$RANDOM_SVD_N_CORES

if (N_CORES > 1) {
  # if we are parallelizing, then disable BLAS parallelization
  options(bigstatsr.check.parallel.blas = FALSE)
  blas_nproc <- getOption("default.nproc.blas")
  options(default.nproc.blas = NULL)
}

archs4_svdRes=big_randomSVD(
  archs4_fbm_filt,
  k = SVD_K_archs4,
  ncores = N_CORES
)

if (N_CORES > 1) {
  # restore previous state
  options(bigstatsr.check.parallel.blas = TRUE)
  options(default.nproc.blas = blas_nproc)
}

In [ ]:
saveRDS(archs4_svdRes, file = file.path(output_dir, "archs4_svdRes.rds"))